In [143]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from shutil import copyfile
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping


In [144]:
desired_width = 224
desired_height = 224
channels = 3
num_classes = 4

# Define the image size and batch size
image_size = (desired_height, desired_width)
# Define a list of possible batch sizes as per dataset size you shoud add bigger sizes
BATCH_SIZES = [8, 16, 32]
# Generate a random index to select a batch size from the list
random_index = np.random.randint(0, len(BATCH_SIZES))

# Select a batch size using the random index
BATCH_SIZE = BATCH_SIZES[random_index]

In [145]:
train_images_folder = 'data/train_images'
test_images_folder = 'data/test_images'

In [146]:
# Read the CSV file
df = pd.read_csv("data/train.csv")
df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [147]:
# Extract the necessary columns
image_ids = df["ImageId"].tolist()
class_ids = df["ClassId"].tolist()
encoded_pixels = df["EncodedPixels"].tolist()

In [148]:
# # Function to extract SURF features from an image
# def extract_surf_features(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     sift = cv2.SIFT_create()
#     _, des = sift.detectAndCompute(gray, None)
#     return des


In [149]:
def preprocess_image(image, desired_width=256, desired_height=256, uni8_required=False):
    # Check if the image is empty
    if image is None:
        print("Error: Image is empty.")
        return None
    
    # Check if the image has the correct depth
    if image.dtype != np.uint8:
        print("Error: Incorrect image depth. Expected uint8.")
        return None
    
    # Resize the image to the desired size
    resized_image = cv2.resize(image, (desired_width, desired_height))
    
    # Normalize the pixel values to a specific range (e.g., 0 to 1)
    normalized_image = resized_image / 255.0
    
    # Apply any additional preprocessing steps here
    # For example, you can perform data augmentation, apply color transformations, etc.
    
    # Convert the image to the appropriate data type (e.g., float32)
    preprocessed_image = np.float32(normalized_image)

    if uni8_required:
        # Convert the image to uint8 
        preprocessed_image = cv2.convertScaleAbs(preprocessed_image * 255.0)
    
    return preprocessed_image


In [150]:
# resize one image
def resize_images(source_dir, target_dir, image_path, target_size):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, target_size)
    _, filename = os.path.split(image_path)
    save_path = os.path.join(target_dir, filename)
    cv2.imwrite(save_path, resized_image)
    return resized_image

In [151]:
# # Load and preprocess your image data
# image = cv2.imread('./images/corrosion_image.png')

# # Check if the image is empty
# if image is None:
#     print("Error: Failed to load the image.")
#     exit()

# # Check the image depth
# if image.dtype != np.uint8:
#     print("Error: Incorrect image depth. Expected uint8.")
#     exit()

# # Preprocess the image
# image = preprocess_image(image, desired_width, desired_height, uni8_required=True)
# print(image.dtype)

In [152]:
# cv2.imshow('corrosion_image', image)
# cv2.waitKey()

In [153]:
# # Check if the preprocessed image is None
# if image is None:
#     print("Error: Failed to preprocess the image.")
#     exit()

# # Check the image depth
# if image.dtype != np.uint8:
#     print("Error: Incorrect image depth. Expected uint8.")
#     exit()


# print(image.dtype)

# # Extract SURF features from the preprocessed image
# surf_features = extract_surf_features(image)

In [154]:
# Function to resize an image
def resize_test_image(image_path, target_size):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, target_size)
    return resized_image

# Iterate over the test images
for image_file in os.listdir(test_images_folder):
    # Check if the file is an image
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Construct the path to the image file
        image_path = os.path.join(test_images_folder, image_file)

        # Resize the image
        resized_image = resize_test_image(image_path, image_size)

        # Save the resized image
        resized_image_path = os.path.join(test_images_folder, f"{image_file}")
        cv2.imwrite(resized_image_path, resized_image)

        print(f"Resized image saved: {resized_image_path}")


Resized image saved: data/test_images/resized_0042e163f.jpg
Resized image saved: data/test_images/resized_002451917.jpg
Resized image saved: data/test_images/resized_000ccc2ac.jpg
Resized image saved: data/test_images/resized_0000f269f.jpg
Resized image saved: data/test_images/resized_0098ca44e.jpg
Resized image saved: data/test_images/resized_006f39c41.jpg
Resized image saved: data/test_images/resized_004f40c73.jpg
Resized image saved: data/test_images/resized_003c5da97.jpg
Resized image saved: data/test_images/resized_00513039a.jpg
Resized image saved: data/test_images/resized_008725cbc.jpg


In [155]:
# Create a dictionary to store class folders
class_folders = {}

# Iterate over the dataset
for image_id, class_id, encoded_pixel in zip(image_ids, class_ids, encoded_pixels):
    # Construct the path to the image file
    image_path = os.path.join(train_images_folder, image_id)

    # Check if the source file exists
    if not os.path.exists(image_path):
        print(f"Warning: Image file '{image_path}' not found. Skipping...")
        continue

    print(image_path)
    
    # Resize the image
    target_folder = os.path.join(train_images_folder, str(class_id))
    resized_image = resize_images(train_images_folder, target_folder, str(image_path), image_size)
    
    # Perform any additional preprocessing steps here
    # preprocessed_image = preprocess_image(resized_image)  # replace preprocess_image with your own preprocessing function

# Split the dataset into train and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    image_ids, class_ids, test_size=0.2, random_state=42
)

In [156]:
train_images = []
test_images = []

In [157]:
# Preprocess train images
for filename in os.listdir(train_images_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(train_images_folder, filename)
        image = cv2.imread(image_path)
        preprocessed_image = preprocess_image(image, desired_width, desired_height)
        train_images.append(preprocessed_image)

train_images = np.array(train_images)

In [158]:
# Preprocess test images
for filename in os.listdir(test_images_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(test_images_folder, filename)
        image = cv2.imread(image_path)
        preprocessed_image = preprocess_image(image, desired_width, desired_height)
        test_images.append(preprocessed_image)

test_images = np.array(test_images)

In [159]:
# Print the shape of the preprocessed images
print("Train images shape:", train_images.shape)
print("Test images shape:", test_images.shape)

Train images shape: (0,)
Test images shape: (10, 224, 224, 3)


In [160]:
# Create an instance of ImageDataGenerator for the test set
test_datagen = ImageDataGenerator(
    rescale=1./255   # Rescale the pixel values to the range [0, 1]
)

# Create an instance of ImageDataGenerator for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,      # Rescale the pixel values to the range [0, 1]
    rotation_range=20,   # Randomly rotate the images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift the images horizontally by up to 20% of the image width
    height_shift_range=0.2, # Randomly shift the images vertically by up to 20% of the image height
    shear_range=0.2,         # Randomly shear the images by up to 20%
    zoom_range=0.3,          # Randomly zoom the images by up to 20%
    horizontal_flip=True,    # Randomly flip the images horizontally
    vertical_flip=True,
    brightness_range = (0.8, 1.2), # brightness adjuster so that some partial images can detect when ground is dark
    fill_mode='nearest',      # Fill any empty pixels with the nearest available pixel value
    featurewise_center=False,
    featurewise_std_normalization=False,
    data_format='channels_last',
    validation_split=0.2
)

# Normalize the data
train_datagen.mean = [123.68, 116.779, 103.939] # RGB channel mean values from ImageNet
train_datagen.std = [58.393, 57.12, 57.375]

# Use the flow_from_directory method to load the images from the directory
train_generator = train_datagen.flow_from_directory(
    train_images_folder,                 # Path to the training set directory
    target_size=image_size,    # Resize the images to 256x256
    batch_size=BATCH_SIZE,             # Use batches of 32 images
    class_mode='categorical',   # Use categorical cross-entropy loss
    subset='training',
    shuffle=True  # Shuffle the files from different folders
)

# Load the validation data into x_val and y_val
validation_generator = train_datagen.flow_from_directory(
    directory=train_images_folder,
    target_size=image_size,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Load the test data into x_test and y_test
test_generator = test_datagen.flow_from_directory(
    directory=test_images_folder,
    target_size=image_size,
    batch_size=BATCH_SIZE//2,
    class_mode='categorical'
)

# Handle any exceptions that may occur during data generation
try:
    x_train, y_train = next(train_generator)
except Exception as e:
    print(f"Error during training data generation: {e}")
    
try:
    x_val, y_val = next(validation_generator)
except Exception as e:
    print(f"Error during validation data generation: {e}")

try:
    x_test, y_test = next(test_generator)
except Exception as e:
    print(f"Error during test data generation: {e}")

Found 50 images belonging to 4 classes.
Found 9 images belonging to 4 classes.
Found 0 images belonging to 0 classes.


In [161]:
# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(desired_height, desired_width, channels))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [162]:
# Create a new model and add the pre-trained VGG16 base
model = Sequential()
model.add(base_model)

# Add custom layers for classification
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [163]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_5 (Flatten)         (None, 25088)             0         
                                                                 
 dense_10 (Dense)            (None, 256)               6422784   
                                                                 
 dense_11 (Dense)            (None, 4)                 1028      
                                                                 
Total params: 21138500 (80.64 MB)
Trainable params: 6423812 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [164]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', verbose=1)

In [165]:
# Train the model with early stopping
history = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[early_stopping])

Epoch 1/100
2/2 [==============================] - 2s 1s/step - loss: 4.0926 - accuracy: 0.2600 - val_loss: 2.9410 - val_accuracy: 0.6667
Epoch 2/100
2/2 [==============================] - 1s 354ms/step - loss: 4.8973 - accuracy: 0.5400 - val_loss: 3.8895 - val_accuracy: 0.2222
Epoch 3/100
2/2 [==============================] - 1s 628ms/step - loss: 4.9712 - accuracy: 0.3400 - val_loss: 2.3689 - val_accuracy: 0.6667
Epoch 4/100
2/2 [==============================] - 1s 356ms/step - loss: 4.6903 - accuracy: 0.5400 - val_loss: 2.0837 - val_accuracy: 0.6667
Epoch 5/100
2/2 [==============================] - 1s 553ms/step - loss: 2.7872 - accuracy: 0.5800 - val_loss: 1.5289 - val_accuracy: 0.6667
Epoch 6/100
2/2 [==============================] - 1s 581ms/step - loss: 2.2055 - accuracy: 0.2600 - val_loss: 2.1491 - val_accuracy: 0.0000e+00
Epoch 6: early stopping


In [166]:
# Check if the desired accuracy and validation accuracy thresholds are met
if history.history['accuracy'][-1] >= 0.95 and history.history['val_accuracy'][-1] >= 0.8:
    print("Early stopping criteria met!")

In [169]:
# Iterate over the test images
for image_file in os.listdir(test_images_folder):
    # Load and preprocess the test image
    image_path = os.path.join(test_images_folder, image_file)
    if image_file.endswith(".jpg") or image_file.endswith(".png"):
        test_image = cv2.imread(image_path)
        
        if test_image is None:
            print(f"Error loading image: {image_path}")
            continue
        
        # Reshape the test image
        test_image_input = np.expand_dims(test_image, axis=0)
        
        # Make predictions on the test image
        predictions = model.predict(test_image_input)
        
        # Get the predicted class label
        predicted_class_label = np.argmax(predictions)
        
        # Print the predicted class label for the test image
        print(f"Test image: {image_file} - Predicted class label: {predicted_class_label}")


1/1 [==============================] - 0s 19ms/step
Test image: resized_0042e163f.jpg - Predicted class label: 1
1/1 [==============================] - 0s 17ms/step
Test image: resized_002451917.jpg - Predicted class label: 1
1/1 [==============================] - 0s 16ms/step
Test image: resized_000ccc2ac.jpg - Predicted class label: 1
1/1 [==============================] - 0s 16ms/step
Test image: resized_0000f269f.jpg - Predicted class label: 2
1/1 [==============================] - 0s 18ms/step
Test image: resized_0098ca44e.jpg - Predicted class label: 1
1/1 [==============================] - 0s 18ms/step
Test image: resized_006f39c41.jpg - Predicted class label: 1
1/1 [==============================] - 0s 18ms/step
Test image: resized_004f40c73.jpg - Predicted class label: 1
1/1 [==============================] - 0s 18ms/step
Test image: resized_003c5da97.jpg - Predicted class label: 2
1/1 [==============================] - 0s 17ms/step
Test image: resized_00513039a.jpg - Predicte